In [1]:
import os
import sys

sys.dont_write_bytecode = True
os.environ["NUMBA_DISABLE_JIT"] = "1"


In [2]:
import numpy as np
import pandas as pd
from my_stuff import MufexKeys
from nb_quantfreedom.exchanges.mufex_exchange.mufex import Mufex
from nb_quantfreedom.nb_enums import DynamicOrderSettingsArrays
from nb_quantfreedom.utils import pretty_qf
from requests import get
from talib import RSI
np.set_printoptions(formatter={"float_kind": "{:.2f}".format})

pd.options.display.float_format = "{:.2f}".format

mufex_main = Mufex(
    api_key=MufexKeys.api_key,
    secret_key=MufexKeys.secret_key,
    use_test_net=False,
)

%load_ext autoreload
%autoreload 2


In [3]:
candles = mufex_main.get_candles_df(
    symbol="BTCUSDT",
    timeframe="30m",
    limit=200
)
pd.DataFrame(candles)

It took 00 mins and 00 seconds to download 200 candles


,0,1,2,3,4
0,1697484600000.00,28572.40,28577.30,28380.90,28452.80
1,1697486400000.00,28452.80,28633.60,28418.70,28586.50
2,1697488200000.00,28586.50,28586.50,28305.10,28390.60
3,1697490000000.00,28390.60,28404.00,28221.30,28348.50
4,1697491800000.00,28348.50,28442.50,28348.50,28411.80
...,...,...,...,...,...
195,1697835600000.00,29565.80,29628.30,29560.00,29593.40
196,1697837400000.00,29593.40,29624.50,29581.90,29596.60
197,1697839200000.00,29596.60,29640.70,29574.60,29609.00
198,1697841000000.00,29609.00,29659.80,29593.90,29649.60


In [4]:
closing_p = candles[:, 4]
up = np.zeros(closing_p.size)
down = np.zeros(closing_p.size)
mean_up = np.zeros(closing_p.size)
mean_down = np.zeros(closing_p.size)
rsi = np.full(closing_p.size, np.nan)

In [5]:
for counter in range(1, 13):
    pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
    if pchg > 0:
        up[counter] = pchg
    else:
        down[counter] = abs(pchg)


In [6]:
for counter in range(14, 200):
    pchg = (closing_p[counter] - closing_p[counter - 1]) / closing_p[counter - 1]
    if pchg > 0:
        up[counter] = pchg
    else:
        down[counter] = abs(pchg)
    mean_up[counter] = np.mean(up[counter - 14 : counter + 1])
    mean_down[counter] = np.mean(down[counter - 14 : counter + 1])
    rsi[counter] = 100 - 100 / (1 + (mean_up[counter] / mean_down[counter]))

In [7]:
RSI(closing_p, 14)

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 45.14, 41.04, 40.59, 38.55, 36.97, 38.29, 39.78, 35.16, 42.54,
       42.15, 54.08, 52.23, 54.91, 52.21, 54.98, 56.63, 58.30, 55.74,
       52.12, 51.35, 40.41, 42.96, 44.97, 51.08, 55.50, 55.12, 55.17,
       58.24, 62.00, 51.14, 51.17, 51.63, 45.88, 51.09, 56.14, 56.46,
       51.32, 57.36, 52.78, 50.84, 48.76, 49.79, 47.20, 48.98, 45.89,
       43.98, 44.62, 53.58, 53.56, 54.03, 56.75, 53.75, 67.91, 65.72,
       63.79, 63.73, 62.85, 53.83, 48.54, 50.42, 52.81, 52.96, 35.20,
       42.23, 42.85, 41.45, 41.27, 43.87, 41.22, 43.40, 41.59, 39.02,
       34.20, 41.03, 39.70, 45.42, 47.44, 51.46, 46.24, 47.86, 44.54,
       43.23, 41.22, 44.75, 42.26, 44.54, 43.89, 48.01, 51.03, 50.01,
       48.88, 51.33, 55.01, 55.00, 41.28, 37.39, 40.42, 44.01, 45.78,
       45.47, 48.39, 53.62, 50.85, 50.90, 48.46, 45.22, 52.83, 55.21,
       51.58, 55.17, 58.82, 59.48, 64.96, 58.77, 60.88, 58.81, 62.19,
       66.89,

In [8]:
rsi

array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, 45.68, 41.76, 32.95, 39.20, 39.48, 35.70, 38.11, 25.08, 36.06,
       32.33, 55.41, 57.87, 56.37, 53.52, 56.35, 56.38, 64.72, 62.74,
       62.37, 64.40, 51.07, 51.95, 59.18, 58.91, 62.74, 52.59, 54.91,
       54.93, 62.15, 49.06, 47.20, 45.61, 42.80, 50.29, 55.32, 68.81,
       61.31, 64.61, 55.68, 49.36, 47.78, 48.65, 43.32, 40.15, 46.27,
       44.41, 44.42, 58.12, 53.25, 46.96, 49.54, 53.02, 62.11, 66.07,
       66.94, 70.29, 68.60, 62.57, 55.41, 60.39, 64.54, 64.28, 38.67,
       44.82, 44.99, 41.76, 42.98, 26.91, 26.21, 29.46, 28.43, 27.28,
       27.59, 36.70, 33.76, 36.64, 38.38, 60.58, 47.21, 48.19, 46.60,
       45.47, 40.77, 46.90, 42.40, 46.27, 48.69, 59.63, 56.38, 57.69,
       50.15, 49.91, 48.15, 57.38, 39.40, 38.84, 43.57, 49.76, 47.64,
       50.82, 51.09, 56.85, 50.28, 46.84, 45.76, 44.02, 48.60, 47.29,
       44.66, 62.98, 74.25, 72.99, 74.89, 66.56, 68.69, 64.75, 63.78,
       71.41,